In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from copy import deepcopy
from matplotlib import pyplot as plt

import os
import datetime
import time
import json
import pickle
import os
import ast

NAFILL = -999

In [2]:
with open("config.json", "r") as f:
    config = json.load(f)
folder_path = config["data_root_path"]
focus_path = folder_path + "Selection_Group/" + config["label_focus"] + "/"
eps_long_to_short = config["eps_long_to_short"]
epis = config["epis"]
wks = config["wks"]

In [97]:
with open(focus_path + "raw_data_pool.pkl", "rb") as f:
    raw_data_pool = pickle.load(f)
df_phaseI_select = raw_data_pool["data_phaseI"]
df_phaseII_select = raw_data_pool["data_phaseII"]
df_label = raw_data_pool["labels"]
df_label = df_label.set_index("PID")

In [6]:
with open(folder_path + "feature_dis_col_epis.json","r") as f:
    feature_dis_col_epis = json.load(f)
with open(folder_path + "symbol_to_int_map_day.json", "r") as f:
    symbol_to_int_map_day = json.load(f)
with open(folder_path + "int_to_symbol_map_day.json", "r") as f:
    int_to_symbol_map_day = json.load(f)
with open(folder_path + "int_to_feature_map_day.json", "r") as f:
    int_to_feature_map_day = json.load(f)

In [104]:
with open(focus_path + "top_features.json", "r") as f:
    top_features = json.load(f)

In [100]:
PID_list_grp1_phaseI = df_label[df_label["label_I"]].index.tolist()
PID_list_grp2_phaseI = df_label[~df_label["label_I"]].index.tolist()
PID_list_grp1_phaseII = df_label[df_label["label_II"]].index.tolist()
PID_list_grp2_phaseII = df_label[~df_label["label_II"]].index.tolist()

In [117]:
count = 0
for wk in wks:
    for epi in epis:
        count += len(top_features[wk][epi])
print(count)

148


In [110]:
def obtain_unimodal_feature(df_data):
    df_unimodal_asso = {}
    for wk in wks:
        df_unimodal_asso[wk] = {}
        for epi in epis:
            print(wk,epi)
            ruleset1 = pd.read_csv(focus_path + "SPMF_select/" + "asso_rule_straight_" + wk + "_" + epi + ".csv")
            ruleset2 = pd.read_csv(focus_path + "SPMF_select/" + "asso_rule_nostraight_" + wk + "_" + epi + ".csv")
            ruleset = pd.concat([ruleset1, ruleset2])
            ruleset.reset_index(drop = True, inplace = True)
            ruleset["X"] = ruleset["X"].apply(lambda x : ast.literal_eval(x))
            ruleset["Y"] = ruleset["Y"].apply(lambda x : ast.literal_eval(x))

            df_unimodal_asso[wk][epi] = pd.DataFrame()
            df_buf = deepcopy(df_data)

            df_unimodal = df_buf[["PID"] + top_features[wk][epi]]
            df_unimodal_mean = df_unimodal.groupby("PID").mean()
            df_unimodal_mean.columns = ["mean_" + y for y in top_features[wk][epi]]
            df_unimodal_std = df_unimodal.groupby("PID").std()
            df_unimodal_std.columns = ["std_" + y for y in top_features[wk][epi]]
            df_unimodal_mean_std = df_unimodal_mean.merge(df_unimodal_std, how = "outer", left_index=True, right_index=True)
#             df_unimodal_mean_std = deepcopy(df_unimodal_mean)
            df_unimodal_asso[wk][epi] = deepcopy(df_unimodal_mean_std-df_unimodal_mean_std.mean())/df_unimodal_mean_std.std()
#             if (df_unimodal_asso[wk][epi].shape[0] == 0):
#                 df_unimodal_asso[wk][epi] = deepcopy(df_unimodal_mean_std)
#             else:
#                 df_unimodal_asso[wk][epi] = df_unimodal_asso[wk][epi].merge(df_unimodal_mean_std, how = "outer", left_index=True, right_index=True)
#     #         df_unimodal_features[wk][epi] = deepcopy(df_unimodal_asso[wk][epi].columns)

    df_unimodal_asso_combined = pd.DataFrame()
    for wk in wks:
        for epi in epis:
            if (df_unimodal_asso_combined.shape[0] == 0):
                df_unimodal_asso_combined = deepcopy(df_unimodal_asso[wk][epi])
            else:
                df_unimodal_asso_combined = df_unimodal_asso_combined.merge(df_unimodal_asso[wk][epi], how = "outer", left_index=True, right_index=True)
    return df_unimodal_asso_combined

In [130]:
def obtain_contextual_feature_from_rule(df_data):
    df_multimodal_asso = {}
    for wk in wks:
        df_multimodal_asso[wk] = {}
        for epi in epis:
            print(wk,epi)
            ruleset1 = pd.read_csv(focus_path + "SPMF_select/" + "asso_rule_straight_" + wk + "_" + epi + ".csv")
            ruleset2 = pd.read_csv(focus_path + "SPMF_select/" + "asso_rule_nostraight_" + wk + "_" + epi + ".csv")
            ruleset = pd.concat([ruleset1, ruleset2])
            ruleset.reset_index(drop = True, inplace = True)
            ruleset["X"] = ruleset["X"].apply(lambda x : ast.literal_eval(x))
            ruleset["Y"] = ruleset["Y"].apply(lambda x : ast.literal_eval(x))

            df_multimodal_asso[wk][epi] = pd.DataFrame()
            df_buf = deepcopy(df_data)

            for rule_idx, rule in ruleset.iterrows():
                index_flag = True
                for x in rule["X"]:
                    dis_label_col = int_to_symbol_map_day[x][:-2] 
                    dis_label = int_to_symbol_map_day[x][-1]
                    index_flag = index_flag & (df_buf[dis_label_col] == dis_label)
                feature_columns = [int_to_feature_map_day[y] for y in rule["Y"]]
                feature_columns_name = [y + "_" + wk + "_" + epi + "_rule" + str(rule_idx) for y in feature_columns]

                df_multimodal = df_buf[index_flag][["PID"] + feature_columns]
                df_multimodal_mean = df_multimodal.groupby("PID").mean()
                df_multimodal_mean.columns = ["mean_" + y for y in feature_columns_name]
                df_multimodal_std = df_multimodal.groupby("PID").std()
                df_multimodal_std.columns = ["std_" + y for y in feature_columns_name]
                df_multimodal_mean_std = df_multimodal_mean.merge(df_multimodal_std, how = "outer", left_index=True, right_index=True)
    #             df_multimodal_mean_std = deepcopy(df_multimodal_mean)
                df_multimodal_mean_std = (df_multimodal_mean_std-df_multimodal_mean_std.mean())/df_multimodal_mean_std.std()
                if (df_multimodal_asso[wk][epi].shape[0] == 0):
                    df_multimodal_asso[wk][epi] = deepcopy(df_multimodal_mean_std)
                else:
                    df_multimodal_asso[wk][epi] = df_multimodal_asso[wk][epi].merge(df_multimodal_mean_std, how = "outer", left_index=True, right_index=True)
    #         df_multimodal_features[wk][epi] = deepcopy(df_multimodal_asso[wk][epi].columns)

    df_multimodal_asso_combined = pd.DataFrame()
    for wk in wks:
        for epi in epis:
            if (df_multimodal_asso_combined.shape[0] == 0):
                df_multimodal_asso_combined = deepcopy(df_multimodal_asso[wk][epi])
            else:
                df_multimodal_asso_combined = df_multimodal_asso_combined.merge(df_multimodal_asso[wk][epi], how = "outer", left_index=True, right_index=True)
    return df_multimodal_asso_combined

In [121]:
df_unimodal_asso_combined_phaseI = obtain_unimodal_feature(df_phaseI_select)
df_unimodal_asso_combined_phaseII = obtain_unimodal_feature(df_phaseII_select)
df_unimodal_asso_combined_phaseI = df_unimodal_asso_combined_phaseI.loc[df_label.index]
df_unimodal_asso_combined_phaseII = df_unimodal_asso_combined_phaseI.loc[df_label.index]
df_unimodal_asso_combined_phaseI.to_csv(focus_path + "X_unimodal_phaseI.csv")
df_unimodal_asso_combined_phaseII.to_csv(focus_path + "X_unimodal_phaseII.csv")

wkdy mo
wkdy af
wkdy ev
wkdy ni
wkend mo
wkend af
wkend ev
wkend ni
wkdy mo
wkdy af
wkdy ev
wkdy ni
wkend mo
wkend af
wkend ev
wkend ni


In [131]:
df_contextual_asso_combined_phaseI = obtain_contextual_feature_from_rule(df_phaseI_select)
df_contextual_asso_combined_phaseII = obtain_contextual_feature_from_rule(df_phaseII_select)
df_contextual_asso_combined_phaseI = df_multimodal_asso_combined_phaseI.loc[df_label.index]
df_contextual_asso_combined_phaseII = df_multimodal_asso_combined_phaseII.loc[df_label.index]
df_contextual_asso_combined_phaseI.to_csv(focus_path + "X_contextual_phaseI.csv")
df_contextual_asso_combined_phaseII.to_csv(focus_path + "X_contextual_phaseII.csv")

wkdy mo
wkdy af
wkdy ev
wkdy ni
wkend mo
wkend af
wkend ev
wkend ni
wkdy mo
wkdy af
wkdy ev
wkdy ni
wkend mo
wkend af
wkend ev
wkend ni


# Train & Test

In [126]:
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import tree  
from sklearn import svm
from sklearn import ensemble
from sklearn import linear_model
import sklearn
import xgboost

In [141]:
def adaboost(X,X_II,Y,Y_II,n_estimators = 10, max_depth = -1, max_leaf_nodes = -1):
    if (max_depth == -1):
        clf = ensemble.AdaBoostClassifier(n_estimators = n_estimators,
                                          base_estimator = tree.DecisionTreeClassifier(
#                                               max_depth= 5),
                                              max_leaf_nodes = max_leaf_nodes),
                                          learning_rate = 0.001)
    else:
        clf = ensemble.AdaBoostClassifier(n_estimators = n_estimators,
                                          base_estimator = tree.DecisionTreeClassifier(
                                              max_depth= max_depth),
#                                               max_leaf_nodes = max_leaf_nodes),
                                          learning_rate = 0.001)
    results = {}
    iter_round = 0

    XX = deepcopy(X)

    importance_index = range(len(XX.columns))
    while (True):
        iter_round += 1
        print(XX.shape)
        train_acc = []
        test_acc = []
        y_pred_list = []
        y_test_list = []
        loo = LeaveOneOut()

        for train_index, test_index in loo.split(XX):
            X_train, X_test = XX.iloc[train_index,:], XX.iloc[test_index,:]
            Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
            a = clf.fit(X_train,Y_train)
            y_pred_list.append(a.predict(X_test))
            y_test_list.append(Y_test)
            train_acc.append(accuracy_score(a.predict(X_train),Y_train))
            test_acc.append(accuracy_score(a.predict(X_test),Y_test))
            
        # retrain to obtain a subset feature
        confusion_mx = confusion_matrix(y_test_list,y_pred_list)
        results[iter_round] = {}
        results[iter_round]["train_acc"] = np.mean(train_acc)
        results[iter_round]["test_acc"] = np.mean(test_acc)
        results[iter_round]["confusion_matrix"] = deepcopy(confusion_mx)

        a = clf.fit(XX,Y)
        importance = a.feature_importances_
        importance_index_new = np.array([idx for idx, x in enumerate(importance) if x > 0])

        if (len(importance_index_new) == len(importance_index)):
            y_pred_phaseII = a.predict(X_II.fillna(NAFILL)[np.array(XX.columns.tolist())[importance_index_new]])
            results[iter_round]["test_acc_phaseII"] = accuracy_score(Y_II,y_pred_phaseII)
            results[iter_round]["confusion_matrix_phaseII"] = confusion_matrix(Y_II,y_pred_phaseII)
            print("train acc phase1: ", results[iter_round]["train_acc"],
                  "test acc phase1: ", results[iter_round]["test_acc"],
                  "phase2: ", results[iter_round]["test_acc_phaseII"])
            break
        else:
            results[iter_round]["importance_feature"] = deepcopy(np.array(XX.columns.tolist())[importance_index_new])
            print(importance_index_new)
            importance_index = deepcopy(importance_index_new)
            XX = X[np.array(XX.columns.tolist())[importance_index]]
            print("train acc ",np.mean(train_acc), " test acc ", np.mean(test_acc))
    return results

## Unimodal Filtered

In [136]:
X_phaseI_unimodal = pd.read_csv(focus_path + "X_unimodal_phaseI.csv")
X_phaseI_unimodal = X_phaseI_unimodal.set_index("PID")
Y_phaseI = df_label["label_I"]
X_phaseII_unimodal = pd.read_csv(focus_path + "X_unimodal_phaseII.csv")
X_phaseII_unimodal = X_phaseII_unimodal.set_index("PID")
Y_phaseII = df_label["label_II"]

In [133]:
training_unimoal_results_depth = {}
training_unimoal_results_leaf = {}
for n_estimators in [5]:
    training_unimoal_results_depth[n_estimators] = {}
    training_unimoal_results_leaf[n_estimators] = {}
    print("depth")
    for max_depth in [2,3,4]:
        print(n_estimators,max_depth)
        r = adaboost(X_phaseI_unimodal.fillna(NAFILL),X_phaseII_unimodal.fillna(NAFILL),Y_phaseI,Y_phaseII,n_estimators,max_depth=max_depth)
        training_unimoal_results_depth[n_estimators][max_depth] = deepcopy(r)
    print("leaf")
    for max_leaf in [3,4,5,6,7,8]:
        print(n_estimators,max_leaf)
        r = adaboost(X_phaseI_unimodal.fillna(NAFILL),X_phaseII_unimodal.fillna(NAFILL),Y_phaseI,Y_phaseII,n_estimators,max_leaf_nodes=max_leaf)
        training_unimoal_results_leaf[n_estimators][max_leaf] = deepcopy(r)
with open(focus_path + "unimodal_train_result.pkl", "wb") as f:
    pickle.dump({"depth":training_unimoal_results_depth,"leaf":training_unimoal_results_leaf},f)

depth
5 2
(68, 296)
[ 30 126 142 159 174]
train acc  0.883011413520632  test acc  0.47058823529411764
(68, 5)
train acc phase1:  0.8823529411764703 test acc phase1:  0.7794117647058824 phase2:  0.6617647058823529
5 3
(68, 296)
[ 30  36  79  84  98 121 126 142 152 157 159 174 182 187 199]
train acc  0.9664179104477614  test acc  0.5882352941176471
(68, 15)
[ 0  1  2  3  5  6  7  8 10 11 12]
train acc  0.9554433713784022  test acc  0.8235294117647058
(68, 11)
train acc phase1:  0.9554433713784022 test acc phase1:  0.8382352941176471 phase2:  0.7352941176470589
5 4
(68, 296)
[  6  11  29  30  40  98 126 142 149 152 159 169 174 182 259 273]
train acc  0.9914398595258996  test acc  0.5735294117647058
(68, 16)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
train acc  0.9855136084284456  test acc  0.8088235294117647
(68, 15)
[ 0  3  4  5  6  7  9 10 11 12 13 14]
train acc  0.9855136084284456  test acc  0.8088235294117647
(68, 12)
[ 0  1  2  4  5  6  7  8  9 10 11]
train acc  0.985513608428445

### Tricks from the beginning - Out of curiosity

In [155]:
cols_full = [x for x in df_phaseI_select.columns.tolist()[5:] if ("_allday" not in x and "_dis_" not in x)]
df_unimodal_full = df_phaseI_select[["PID"] + cols_full]
df_unimodal_full_mean = df_unimodal_full.groupby("PID").mean()
df_unimodal_full_mean.columns = ["mean_" + y for y in cols_full]
df_unimodal_full_std = df_unimodal_full.groupby("PID").std()
df_unimodal_full_std.columns = ["std_" + y for y in cols_full]
df_unimodal_full_mean_std = df_unimodal_full_mean.merge(df_unimodal_full_std, how = "outer", left_index=True, right_index=True)
df_unimodal_full_mean_std = (df_unimodal_full_mean_std-df_unimodal_full_mean_std.mean())/df_unimodal_full_mean_std.std()
X_phaseI_unimodal_full = deepcopy(df_unimodal_full_mean_std)
X_phaseI_unimodal_full = X_phaseI_unimodal_full.loc[df_label.index]
Y_phaseI = df_label["label_I"]

cols_full = [x for x in df_phaseII_select.columns.tolist()[5:] if ("_allday" not in x and "_dis_" not in x)]
df_unimodal_full = df_phaseII_select[["PID"] + cols_full]
df_unimodal_full_mean = df_unimodal_full.groupby("PID").mean()
df_unimodal_full_mean.columns = ["mean_" + y for y in cols_full]
df_unimodal_full_std = df_unimodal_full.groupby("PID").std()
df_unimodal_full_std.columns = ["std_" + y for y in cols_full]
df_unimodal_full_mean_std = df_unimodal_full_mean.merge(df_unimodal_full_std, how = "outer", left_index=True, right_index=True)
df_unimodal_full_mean_std = (df_unimodal_full_mean_std-df_unimodal_full_mean_std.mean())/df_unimodal_full_mean_std.std()
X_phaseII_unimodal_full = deepcopy(df_unimodal_full_mean_std)
X_phaseII_unimodal_full = X_phaseII_unimodal_full.loc[df_label.index]
Y_phaseII = df_label["label_II"]


In [157]:
training_unimoal_full_results_depth = {}
training_unimoal_full_results_leaf = {}
for n_estimators in [5]:
    training_unimoal_full_results_depth[n_estimators] = {}
    training_unimoal_full_results_leaf[n_estimators] = {}
    print("depth")
    for max_depth in [2,3,4]:
        print(n_estimators,max_depth)
        r = adaboost(X_phaseI_unimodal_full.fillna(NAFILL),X_phaseII_unimodal_full.fillna(NAFILL),Y_phaseI,Y_phaseII,n_estimators,max_depth=max_depth)
        training_unimoal_full_results_depth[n_estimators][max_depth] = deepcopy(r)
    print("leaf")
    for max_leaf in [3,4,5,6,7,8]:
        print(n_estimators,max_leaf)
        r = adaboost(X_phaseI_unimodal_full.fillna(NAFILL),X_phaseII_unimodal_full.fillna(NAFILL),Y_phaseI,Y_phaseII,n_estimators,max_leaf_nodes=max_leaf)
        training_unimoal_full_results_leaf[n_estimators][max_leaf] = deepcopy(r)
with open(focus_path + "unimodal_full_train_result.pkl", "wb") as f:
    pickle.dump({"depth":training_unimoal_full_results_depth,"leaf":training_unimoal_full_results_leaf},f)

depth
5 2
(68, 1688)
[ 435  736  855 1279]
train acc  0.8878402107111499  test acc  0.7205882352941176
(68, 4)
train acc phase1:  0.8825724319578576 test acc phase1:  0.8382352941176471 phase2:  0.6029411764705882
5 3
(68, 1688)
[ 125  212  435  504  640  736  839  855  879  890  926  965 1119 1279
 1677 1684]
train acc  0.9855136084284458  test acc  0.5882352941176471
(68, 16)
[ 1  2  3  5  6  7  8  9 10 12 13 14 15]
train acc  0.9822212467076381  test acc  0.8235294117647058
(68, 13)
[ 0  1  2  3  5  6  7  8  9 10 12]
train acc  0.9822212467076381  test acc  0.8088235294117647
(68, 11)
train acc phase1:  0.9822212467076381 test acc phase1:  0.8088235294117647 phase2:  0.4852941176470588
5 4
(68, 1688)
[  17  284  736  855 1279 1398 1684]
train acc  1.0  test acc  0.6176470588235294
(68, 7)
train acc phase1:  1.0 test acc phase1:  0.8235294117647058 phase2:  0.5441176470588235
leaf
5 3
(68, 1688)
[ 435  736 1279]
train acc  0.8573309920983317  test acc  0.75
(68, 3)
train acc phase1: 

## Contextually Filtered

In [137]:
X_phaseI_contextual = pd.read_csv(focus_path + "X_contextual_phaseI.csv")
X_phaseI_contextual = X_phaseI_contextual.set_index("PID")
Y_phaseI = df_label["label_I"]
X_phaseII_contextual = pd.read_csv(focus_path + "X_contextual_phaseII.csv")
X_phaseII_contextual = X_phaseII_contextual.set_index("PID")
Y_phaseII = df_label["label_II"]

In [95]:
training_contextual_results_depth = {}
training_contextual_results_leaf = {}
for n_estimators in [5]:
    training_contextual_results_depth[n_estimators] = {}
    training_contextual_results_leaf[n_estimators] = {}
    print("depth")
    for max_depth in [2,3,4]:
        print(n_estimators,max_depth)
        r = adaboost(X_phaseI_contextual.fillna(NAFILL),X_phaseII_contextual.fillna(NAFILL),Y_phaseI,Y_phaseII,n_estimators,max_depth=max_depth)
        training_contextual_results_depth[n_estimators][max_depth] = deepcopy(r)
    print("leaf")
    for max_leaf in [3,4,5,6,7,8]:
        print(n_estimators,max_leaf)
        r = adaboost(X_phaseI_contextual.fillna(NAFILL),X_phaseII_contextual.fillna(NAFILL),Y_phaseI,Y_phaseII,n_estimators,max_leaf_nodes=max_leaf)
        training_contextual_results_leaf[n_estimators][max_leaf] = deepcopy(r)
with open(focus_path + "contextual_train_result.pkl", "wb") as f:
    pickle.dump({"depth":training_contextual_results_depth,"leaf":training_contextual_results_leaf},f)

depth
5 2
(68, 316)
[ 37  40  87 214 294]
train acc  0.8037752414398593  test acc  0.5441176470588235
(68, 5)
train acc phase1:  0.7741439859525899 test acc phase1:  0.5882352941176471 phase2:  0.6323529411764706
5 3
(68, 316)
[  3  21  23  37  40  87 127 185 206 268 294]
train acc  0.9161545215100967  test acc  0.6029411764705882
(68, 11)
train acc phase1:  0.9078138718173837 test acc phase1:  0.7058823529411765 phase2:  0.6176470588235294
5 4
(68, 316)
[ 21  23  84  85  87 116 121 149 162 185 268 294]
train acc  0.9839771729587358  test acc  0.5294117647058824
(68, 12)
train acc phase1:  0.9890254609306408 test acc phase1:  0.7205882352941176 phase2:  0.5
leaf
5 3
(68, 316)
[ 37  40  87 294]
train acc  0.7809482001755926  test acc  0.45588235294117646
(68, 4)
train acc phase1:  0.7640474100087797 test acc phase1:  0.5588235294117647 phase2:  0.6323529411764706
5 4
(68, 316)
[ 27  40  87 125 185 268 294]
train acc  0.8628182616330115  test acc  0.5882352941176471
(68, 7)
train acc pha

## Hybrid Features

In [142]:
X_phaseI_hybrid = X_phaseI_unimodal.merge(X_phaseI_contextual, left_index=True, right_index=True)
Y_phaseI = df_label["label_I"]
X_phaseII_hybrid = X_phaseII_unimodal.merge(X_phaseII_contextual, left_index=True, right_index=True)
Y_phaseII = df_label["label_II"]

In [143]:
training_hybrid_results_depth = {}
training_hybrid_results_leaf = {}
for n_estimators in [5]:
    training_hybrid_results_depth[n_estimators] = {}
    training_hybrid_results_leaf[n_estimators] = {}
    print("depth")
    for max_depth in [2,3,4]:
        print(n_estimators,max_depth)
        r = adaboost(X_phaseI_hybrid.fillna(NAFILL),X_phaseII_hybrid.fillna(NAFILL),Y_phaseI,Y_phaseII,n_estimators,max_depth=max_depth)
        training_hybrid_results_depth[n_estimators][max_depth] = deepcopy(r)
    print("leaf")
    for max_leaf in [3,4,5,6,7,8]:
        print(n_estimators,max_leaf)
        r = adaboost(X_phaseI_hybrid.fillna(NAFILL),X_phaseII_hybrid.fillna(NAFILL),Y_phaseI,Y_phaseII,n_estimators,max_leaf_nodes=max_leaf)
        training_hybrid_results_leaf[n_estimators][max_leaf] = deepcopy(r)
with open(focus_path + "hybrid_train_result.pkl", "wb") as f:
    pickle.dump({"depth":training_hybrid_results_depth,"leaf":training_hybrid_results_leaf},f)

depth
5 2
(68, 612)
[126 142 159 174 493]
train acc  0.8893766461808604  test acc  0.5
(68, 5)
train acc phase1:  0.8823529411764703 test acc phase1:  0.7647058823529411 phase2:  0.5588235294117647
5 3
(68, 612)
[ 10  30  36 126 142 159 174 266 336 390 416 432 493 500 571]
train acc  0.9804653204565409  test acc  0.6470588235294118
(68, 15)
[ 1  3  4  5  6  8  9 10 12 13 14]
train acc  0.9734416154521509  test acc  0.7647058823529411
(68, 11)
train acc phase1:  0.9734416154521509 test acc phase1:  0.75 phase2:  0.6323529411764706
5 4
(68, 612)
[104 126 141 159 493 500 570]
train acc  0.9997805092186127  test acc  0.6029411764705882
(68, 7)
[1 2 3 4 5 6]
train acc  1.0  test acc  0.8676470588235294
(68, 6)
train acc phase1:  1.0 test acc phase1:  0.8823529411764706 phase2:  0.6323529411764706
leaf
5 3
(68, 612)
[126 142 493]
train acc  0.8454784899034239  test acc  0.5588235294117647
(68, 3)
train acc phase1:  0.838235294117647 test acc phase1:  0.7794117647058824 phase2:  0.51470588235

In [2]:
57 / 68.0

0.8382352941176471